In [1]:
import pickle
import duckdb
import numpy as np

In [2]:
# 1. Load your Pickle dataset (adjust path)
with open('/home/yaning/Documents/large_files/fires.pkl', 'rb') as f:
    data = pickle.load(f)


In_fires = data["In_fires"]
E_fires = data["E_fires"]
I_fires = data["I_fires"]
Out_fires = data["Out_fires"]

states_list = []
for t in range(In_fires.shape[0]):
    In_states = In_fires[t].view(-1).tolist()
    E_states = E_fires[t].view(-1).tolist()
    I_states = I_fires[t].view(-1).tolist()
    Out_states = Out_fires[t].view(-1).tolist()
    states = In_states + E_states + I_states + Out_states
    states_list.append(states)


In [3]:

# 2. Connect to DuckDB (creates a file)
con = duckdb.connect('/home/yaning/Documents/Spiking_NN/anime/firing_states.duckdb')

# 3. Prepare data for insertion:
# Flatten data into rows (frame, neuron_id, state)
firing_states = []
for t in range(len(states_list)):
    states = states_list[t]
    for n in range(len(states)):
        firing_states.append((t, n, int(states[n])))



In [57]:
firing_states[0]

(0, 0, 0)

In [58]:
con.execute("DROP TABLE IF EXISTS firing_states;")

In [61]:
len(firing_states)

7446000

In [6]:

# 4. Create table
con.execute("""
CREATE TABLE IF NOT EXISTS firing_states (
    frame INTEGER,
    idx INTEGER,
    state INTEGER
);
""")

# 5. Insert data efficiently
con.executemany("INSERT INTO firing_states VALUES (?, ?, ?);", firing_states[:10])

print("Data imported to DuckDB successfully!")


Data imported to DuckDB successfully!


In [8]:
import os

db_file = 'anime/firing_states.duckdb'

if os.path.exists(db_file):
    os.remove(db_file)
    print(f"{db_file} deleted successfully.")
else:
    print(f"{db_file} does not exist.")

anime/firing_states.duckdb deleted successfully.
